# Assignment for Week 3

## 1. Importing necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 2. Importing BeautifulSoup 

In [2]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests

## 3. Scraping Data from Wikipedia

In [3]:
from bs4 import BeautifulSoup
import requests

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url, 'lxml')
soup = BeautifulSoup(req.content, 'html.parser')
table = soup.find("table")

### 3.1 Using a function to parse the HTML table for insertion in a data frame.

In [4]:
def tableDataText(table):    
    """Parses a html segment started with tag <table> followed 
    by multiple <tr> (table rows) and inner <td> (table data) tags. 
    It returns a list of rows with inner columns. 
    Accepts only one <th> (table header/data) in the first row.
    """
    def rowgetDataText(tr, coltag='td'): # td (data) or th (header)       
        return [td.get_text(strip=True) for td in tr.find_all(coltag)]  
    rows = []
    trs = table.find_all('tr')
    headerow = rowgetDataText(trs[0], 'th')
    if headerow: # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs: # for every table row
        rows.append(rowgetDataText(tr, 'td') ) # data row       
    return rows

### 3.2 Creation of the required dataframe 'pd_toronto'

In [5]:
list_table = tableDataText(table)
pd_toronto = pd.DataFrame(list_table[1:], columns=list_table[0])
pd_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
#pd_toronto_ref = pd_toronto[ pd_toronto['Borough'] != 'Not assigned' ]

## 4 Drop Columns where Borough is Not Assigned.

In [7]:
pd_toronto = pd_toronto[ pd_toronto['Borough'] != 'Not assigned' ]

#pd_toronto = pd_toronto.drop(pd_toronto[ pd_toronto['Borough'] == 'Not assigned' ] , axis=0, inplace=True)
#df.drop(df[condition].index, axis=0, inplace=True)
#data = data.drop("Ireland", axis=0). # Delete all rows with label "Ireland"

## 5 Combining Duplicate Neighbourhood Based on Post Code

In [8]:
pd_toronto = pd_toronto.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()

## 6 Updating Neighbourhoods with Value 'Not Assigned' to Bourough 

In [9]:
pd_toronto.loc[pd_toronto.head(500)['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = pd_toronto['Borough']

## 7 Verifying Shape of Datframe

In [11]:
pd_toronto.shape

(103, 3)